In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import tokenize
import nltk
import sudachipy
from sudachipy import tokenizer
from sudachipy import dictionary

#nltk.download('popular')

In [2]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

Using custom data configuration copenlu--nlp_course_tydiqa-cceecfb5416d988a
Found cached dataset parquet (/Users/dpr577/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-cceecfb5416d988a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
len(train_set)

116067

In [3]:
print(len(validation_set))
df_val = pd.DataFrame(validation_set)
df_val = df_val[df_val.language.isin(['finnish', 'english', 'japanese'])]
len(df_val)

13325


3712

In [4]:
df_train = pd.DataFrame(train_set)
df_train.head()

,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
2,వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,వేప,telugu,"{'answer_start': [12], 'answer_text': ['Azadir...","వేప (లాటిన్ Azadirachta indica, syn. Melia aza...",https://te.wikipedia.org/wiki/%E0%B0%B5%E0%B1%...
3,চেঙ্গিস খান কোন বংশের রাজা ছিলেন ?,চেঙ্গিজ খান,bengali,"{'answer_start': [414], 'answer_text': ['বোরজি...",চেঙ্গিজ খান (মঙ্গোলীয়: Чингис Хаан আ-ধ্ব-ব: ...,https://bn.wikipedia.org/wiki/%E0%A6%9A%E0%A7%...
4,రెయ్యలగడ్ద గ్రామ విస్తీర్ణత ఎంత?,రెయ్యలగడ్ద,telugu,"{'answer_start': [259], 'answer_text': ['27 హె...","రెయ్యలగడ్ద, విశాఖపట్నం జిల్లా, గంగరాజు మాడుగుల...",https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B1%...


In [5]:
df_train = df_train[df_train.language.isin(['finnish', 'english', 'japanese'])]
len(df_train)

29868

In [6]:
def get_lang_df(df, language):
    return df[df['language'] == language]

df_train_FI = get_lang_df(df_train, 'finnish').copy()
df_train_JAP = get_lang_df(df_train, 'japanese').copy()
df_train_EN = get_lang_df(df_train, 'english').copy()

In [7]:
df_train_JAP.iloc[0]

question_text                               “ダン” ダニエル・ジャドソン・キャラハンの出身はどこ
document_title                                             ダニエル・J・キャラハン
language                                                       japanese
annotations           {'answer_start': [35], 'answer_text': ['カリフォルニ...
document_plaintext    “ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...
document_url          https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
Name: 1, dtype: object

In [17]:
def tokenize_EN(df):
    df['tokens'] = df['question_text'].apply(word_tokenize)

def tokenize_FI(df):
    df['tokens'] = df['question_text'].apply(word_tokenize, language='finnish')

def helper_func_JAP(question):
    tokenizer_obj = dictionary.Dictionary().create()
    res_list = tokenizer_obj.tokenize(question)
    return [x.surface() for x in res_list]
   
    #return 

def tokenize_JAP(df):
    df['tokens'] = df['question_text'].apply(helper_func_JAP)


tokenize_EN(df_train_EN)
tokenize_FI(df_train_FI)
#tokenize_JAP(df_train_JAP)


In [13]:
test = "“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ"
#tokenizer_obj = dictionary.Dictionary().create()
#res = tokenizer_obj.tokenize(test)
#type(res[0].surface())
helper_func_JAP(test)

['“', 'ダン', '”', ' ', 'ダニエル', '・', 'ジャドソン', '・', 'キャラハン', 'の', '出身', 'は', 'どこ']

In [18]:
def get_most_common_first_n_tokens(df, n):
    df['first_token'] = df['tokens'].apply(lambda x: x[0])
    return df.first_token.value_counts()[:n]

def get_most_common_last_n_tokens(df, n):
    df['last_token'] = df['tokens'].apply(lambda x: x[-1] if x[-1].isalpha() else x[-2])
    return df.last_token.value_counts()[:n]

#For English
df_res = get_most_common_first_n_tokens(df_train_EN, 10)
get_most_common_last_n_tokens(df_train_EN, 10)

born           342
founded        204
die            122
have           104
formed         100
established     96
air             82
released        80
live            76
introduced      72
Name: last_token, dtype: int64

In [19]:
#Finnish
get_most_common_first_n_tokens(df_train_FI, 10)
get_most_common_last_n_tokens(df_train_FI, 10)

syntyi          1072
on               723
kuoli            720
tarkoittaa       488
perustettu       476
syntynyt         398
oli              382
perustettiin     351
sijaitsee        258
pinta-ala        214
Name: last_token, dtype: int64

In [23]:
#Japanese
get_most_common_first_n_tokens(df_train_JAP, 10)


日本         354
『          306
世界          94
ジョン         58
第           56
アメリカ合衆国     54
「           50
アメリカ        50
ウィリアム       44
ジョージ        44
Name: first_token, dtype: int64

In [24]:
get_most_common_last_n_tokens(df_train_JAP, 10)

た     2115
か     1305
何     1192
いつ     996
は      932
どこ     884
誰      746
ある     174
だれ      64
いる      42
Name: last_token, dtype: int64